# Grafici da CSV a Batch (Response Time vs Lambda)

Questo notebook legge i CSV con colonne `SCOPE, ARRIVAL_RATE, BATCH_NUM, MEAN_RESPONSE_TIME, ...`,
calcola media e IC 95% (t-Student) **tra i batch** per ogni `(λ, scope)` e disegna il grafico 2×2 Response Time vs λ.


In [3]:
from pathlib import Path
from glob import glob
from typing import Optional, Sequence, Tuple, Union

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

_T95 = {
    1:12.706, 2:4.303, 3:3.182, 4:2.776, 5:2.571, 6:2.447, 7:2.365, 8:2.306, 9:2.262,
    10:2.228, 11:2.201, 12:2.179, 13:2.160, 14:2.145, 15:2.131, 16:2.120, 17:2.110,
    18:2.101, 19:2.093, 20:2.086, 21:2.080, 22:2.074, 23:2.069, 24:2.064, 25:2.060,
    26:2.056, 27:2.052, 28:2.048, 29:2.045, 30:2.042
}
def tcrit95(n: int) -> float:
    df = max(1, int(n) - 1)
    return _T95.get(min(df, 30), 1.96)

def _resolve_csvs(path_or_glob: Union[str, Path]) -> list[Path]:
    p = str(path_or_glob)
    if any(ch in p for ch in "*?[]"):
        matches = [Path(m) for m in glob(p)]
        if not matches:
            raise FileNotFoundError(f"Nessun file per il pattern: {p}")
        matches.sort(key=lambda m: m.stat().st_mtime)
        return matches
    pth = Path(p)
    if not pth.exists():
        raise FileNotFoundError(f"File inesistente: {pth}")
    return [pth]

REQUIRED_COLS = {"SCOPE","ARRIVAL_RATE","BATCH_NUM","MEAN_RESPONSE_TIME"}

def load_batches(csv_input: Union[str, Path]) -> pd.DataFrame:
    files = _resolve_csvs(csv_input)
    dfs = []
    for f in files:
        d = pd.read_csv(f)
        d.columns = [c.strip().upper() for c in d.columns]
        dfs.append(d)
    df = pd.concat(dfs, ignore_index=True)
    missing = REQUIRED_COLS - set(df.columns)
    if missing:
        raise KeyError(f"Mancano colonne {sorted(missing)} nel CSV")
    df["ARRIVAL_RATE"] = pd.to_numeric(df["ARRIVAL_RATE"], errors="coerce")
    df["BATCH_NUM"]    = pd.to_numeric(df["BATCH_NUM"],    errors="coerce")
    df["MEAN_RESPONSE_TIME"] = pd.to_numeric(df["MEAN_RESPONSE_TIME"], errors="coerce")
    df["SCOPE"] = df["SCOPE"].astype(str).str.upper()
    return df.dropna(subset=["ARRIVAL_RATE","BATCH_NUM","MEAN_RESPONSE_TIME","SCOPE"])    

def summarize_rt_by_lambda_scope(df: pd.DataFrame) -> pd.DataFrame:
    recs = []
    for (lam, scope), g in df.groupby(["ARRIVAL_RATE","SCOPE"]):
        y = g["MEAN_RESPONSE_TIME"].to_numpy(dtype=float)
        n = int(len(y))
        mean = float(np.mean(y)) if n>0 else np.nan
        s    = float(np.std(y, ddof=1)) if n>1 else 0.0
        se   = s/np.sqrt(n) if n>0 else np.nan
        hw   = tcrit95(n)*se if n>0 else np.nan
        lo, hi = mean - hw, mean + hw
        recs.append({
            "ARRIVAL_RATE": lam, "SCOPE": scope,
            "N_BATCH": n, "MEAN": mean,
            "SE": se, "HALF_WIDTH": hw,
            "CI_LO": lo, "CI_HI": hi,
        })
    out = pd.DataFrame.from_records(recs)
    return out.sort_values(["SCOPE","ARRIVAL_RATE"]).reset_index(drop=True)

def plot_rt_vs_lambda_from_batches(
    csv_input: Union[str, Path],
    scopes: Sequence[str] = ("OVERALL","A","B","P"),
    lambda_star: Optional[float] = None,
    analytic_df: Optional[pd.DataFrame] = None,
    save_path: Optional[Union[str, Path]] = None,
    show: bool = True
):
    df = load_batches(csv_input)
    summary = summarize_rt_by_lambda_scope(df)

    scope_map = {
        "OVERALL":"OVERALL",
        "A":"A", "B":"B", "P":"P",
        "NODE_A":"A", "NODE_B":"B", "NODE_P":"P",
        "SYSTEM":"OVERALL"
    }
    summary["SCOPE"] = summary["SCOPE"].map(lambda s: scope_map.get(s.upper(), s.upper()))

    fig, axs = plt.subplots(2, 2, figsize=(11, 7))
    axs = axs.ravel()

    handles_ref = None
    labels_ref  = None

    for i, sc in enumerate(scopes[:4]):
        ax = axs[i]
        dsc = summary[summary["SCOPE"] == sc]
        if dsc.empty:
            ax.text(0.5,0.5,f"No data for '{sc}'", ha="center", va="center", transform=ax.transAxes)
            ax.set_axis_off()
            continue

        x  = dsc["ARRIVAL_RATE"].to_numpy()
        y  = dsc["MEAN"].to_numpy()
        hw = dsc["HALF_WIDTH"].to_numpy()

        eb = ax.errorbar(x, y, yerr=hw, fmt="o-", linewidth=1.5, capsize=3, label="simulation (mean)")
        ax.fill_between(x, y-hw, y+hw, alpha=0.20, label="simulation CI band")

        if lambda_star is not None:
            ax.axvline(lambda_star, linestyle="--", linewidth=1.2, alpha=0.8, label="λ*")

        if analytic_df is not None:
            d_ana = analytic_df.copy()
            d_ana.columns = [c.strip().upper() for c in d_ana.columns]
            if {"SCOPE","ARRIVAL_RATE","MEAN_ANALYTIC"} <= set(d_ana.columns):
                d_ana["SCOPE"] = d_ana["SCOPE"].astype(str).str.upper().map(lambda s: scope_map.get(s, s))
                a = d_ana[d_ana["SCOPE"] == sc].sort_values("ARRIVAL_RATE")
                if not a.empty:
                    ax.plot(a["ARRIVAL_RATE"], a["MEAN_ANALYTIC"], "--", linewidth=1.5, label="analytical model")

        ax.set_title(f"{'SYSTEM' if sc=='OVERALL' else sc} average response time")
        ax.set_xlabel("Lambda")
        ax.set_ylabel("Avg response time [s]")
        ax.grid(True, alpha=0.25)
        if handles_ref is None:
            handles_ref, labels_ref = ax.get_legend_handles_labels()

    if handles_ref:
        fig.legend(handles_ref, labels_ref, loc="center left", bbox_to_anchor=(1.02, 0.5), framealpha=0.9)
        fig.subplots_adjust(right=0.82)

    fig.suptitle("Simulation vs Analytical — Response Time", y=0.98)

    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, bbox_inches="tight", dpi=300)

    if show:
        plt.show()
    else:
        plt.close(fig)

    summary
    

In [7]:
# === Parametri rapidi ===
CSV_INPUT = ".output_simulation/obj2_arrivals.csv"  # o un singolo CSV
LAM_STAR  = 1.25                              # opzionale: linea verticale
SAVE_PNG  = None # None per non salvare

_ = plot_rt_vs_lambda_from_batches(CSV_INPUT, lambda_star=LAM_STAR, save_path=SAVE_PNG, show=True)
print("Salvato:", SAVE_PNG)


FileNotFoundError: File inesistente: .output_simulation\obj2_arrivals.csv